# Installer les dépendances nécessaires

In [2]:
!pip install fastapi uvicorn tensorflow pillow scikit-learn streamlit nest-asyncio

# Importer les bibliothèques nécessaires


In [3]:
import os
import tensorflow as tf
import numpy as np
from io import BytesIO
from PIL import Image
import requests
from sklearn.model_selection import train_test_split
from fastapi import FastAPI, UploadFile, File, HTTPException
import uvicorn
import nest_asyncio
import threading
import streamlit as st

# Définir l'API FastAPI


In [4]:
app = FastAPI(
    title="Image Classification API",
    description="API for training and predicting image classification models",
    version="1.0.0"
)

def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        label = 'chien' if 'chien' in filename else 'chat'
        img_path = os.path.join(folder, filename)
        image = Image.open(img_path)
        images.append(np.array(image.resize((128, 128))))
        labels.append(label)
    return np.array(images), np.array(labels)

@app.post("/training")
async def train_model():
    try:
        images, labels = load_images_from_folder('img')

        label_to_index = {'chien': 0, 'chat': 1}
        labels = np.array([label_to_index[label] for label in labels])

        X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

        model = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(128, 128, 3)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(2, activation='softmax')
        ])

        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
        model.save('model/model.h5')
        return {"message": "Model trained successfully"}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    try:
        image = Image.open(BytesIO(file.file.read()))
        image = np.array(image.resize((128, 128)))[np.newaxis, ...]

        model = tf.keras.models.load_model('model/model.h5')
        prediction = model.predict(image)
        predicted_class = np.argmax(prediction, axis=1)[0]

        index_to_label = {0: 'chien', 1: 'chat'}
        predicted_label = index_to_label[predicted_class]

        return {"prediction": predicted_label}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))


# Démarrer l'API FastAPI


In [5]:
def run_api():
    if not os.path.exists('model'):
        os.makedirs('model')
    nest_asyncio.apply()
    uvicorn.run(app, host='0.0.0.0', port=8000)

api_thread = threading.Thread(target=run_api)
api_thread.start()

Exception in thread Thread-11 (run_api):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-75be35038290>", line 5, in run_api
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 40, in _get_event_loop
    loop = events.get_event_loop_policy().get_event_loop()
  File "/usr/lib/python3.10/asyncio/events.py", line 656, in get_event_loop
    raise RuntimeError('There is no current event loop in thread %r.'
RuntimeError: Th

# Définir l'application Streamlit


In [6]:
def run_streamlit():
    st.title("Image Classification App")

    st.write("Upload an image of a dog or a cat to classify it.")

    uploaded_file = st.file_uploader("Choose an image...", type="jpg")

    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption='Uploaded Image.', use_column_width=True)

        if st.button('Predict'):
            files = {'file': uploaded_file.getvalue()}
            response = requests.post("http://localhost:8000/predict", files=files)
            prediction = response.json()
            st.write(prediction)

    if st.button('Download Model'):
        with open("model/model.h5", "rb") as file:
            st.download_button(label="Download Model", data=file, file_name="model.h5")

# Démarrer l'application Streamlit

In [7]:
streamlit_thread = threading.Thread(target=run_streamlit)
streamlit_thread.start()

2024-07-04 14:41:39.673 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
